## ChromaDB 만들고 답변 가지고 오기

---
### Chromadb
- embedding model : 'all-MiniLM-L6-v2'
- Document(page_content = 태그 , metadata = 나머지 정보들)

In [1]:
!pip install chromadb


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import chromadb

client = chromadb.PersistentClient(path='./chromadb_tag_only')

In [3]:
!pip install sentence-transformers


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [65]:
collection = client.get_or_create_collection(name='tourabc')

In [7]:
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 137.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
# pd 생성 후 태그만 임베딩하기
import pandas as pd

df = pd.read_csv('찐찐찐찐막 전국 데이터.csv')

C:\Users\brain\AppData\Local\Temp\ipykernel_10352\3822900607.py:4: DtypeWarning: Columns (2,55,57,58,59,60,63,64,67,68,69,70,72) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('찐찐찐찐막 전국 데이터.csv')


In [2]:
# 메타 데이터 생성
metadata=[]
for idx in range(len(df)):
    meta_data = {}
    meta_data['text'] = df.iloc[idx]['태그']
    meta_data['region'] = df.iloc[idx]['주소'].split()[0]
    meta_data['place'] = df.iloc[idx]['place']
    metadata.append(meta_data)

In [3]:
metadata, len(metadata)

([{'text': '조선의 근본을 상징하는 국왕의 제사터 관광지 문화유적 문화유적돌아보기 서울 서울관광지 서울역사관광 역사관광',
   'region': '광주광역시',
   'place': '사직단'},
  {'text': '그날의 역사와 의의를 자세히 알 수 있는 곳 5.18 518민주화운동기록관 5·18민주화운동기록관 광주 광주여행 기록관',
   'region': '광주광역시',
   'place': '5·18민주화운동 기록관'},
  {'text': '봄을 먼저 맞이하는 곳 광주가볼만한곳 광주꽃구입 광주꽃단지 광주꽃시장 광주나무구입 광주여행',
   'region': '광주광역시',
   'place': '상무화훼단지'},
  {'text': '생산자와 소비자를 보호하는 호남 최대의 도매시장 농수산물 농수산물검사소 농수산물경매 도매시장 서부농수산물도매시장',
   'region': '광주광역시',
   'place': '서부농수산물도매시장'},
  {'text': '코스모스와 억새 물결이 일렁이는 공원 관광지 광주가볼만한곳 광주코스모스 산동교 산동교친수공원 유채꽃명소',
   'region': '광주광역시',
   'place': '산동교친수공원'},
  {'text': '우리나라 최초의 폐선부지 공원 2022친환경추천여행지 관광지 광주 광주_여행지_추천 광주가볼만한곳',
   'region': '광주광역시',
   'place': '푸른길분수공원'},
  {'text': '일본통신사 행차를 기록한 조형선생을 기리기 위한 비 관광지 광주취병조형유허비 교과서속여행 문화재 역사 역사공부 역사를품은곳',
   'region': '광주광역시',
   'place': '취병조형유허비'},
  {'text': '우리나라 무덤의 발달사와 한일관계의 역사를 보여주는 삼국시대 고분 관광지 광주고분 광주월계동장고분 신라고분 역사 역사공부 역사관광지',
   'region': '광주광역시',
   'place': '월계동장고분'},
  {'text': '석불입상과 함께

In [19]:
!pip install -U langchain-community


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [21]:
!pip install langchain-community langchain-core


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
documents = [df.iloc[idx]['태그'] for idx in range(len(df))]

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
docs = text_splitter.create_documents(documents,metadata)
doc = text_splitter.split_documents(docs)

embedding_model = HuggingFaceBgeEmbeddings(model_name = 'all-MiniLM-L6-v2')

vectorstore = Chroma.from_documents(docs, embedding_model)

C:\Users\brain\AppData\Local\Temp\ipykernel_10352\1486395944.py:11: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(model_name = 'all-MiniLM-L6-v2')


In [10]:
query = "부산에 부모님을 모시고 경치 좋은 곳 추천해줘"

results = vectorstore.similarity_search(query,k=10)
print(results[0])

print(vectorstore.similarity_search_with_score(query,k=10))

page_content='명지계곡 물이 흐르는 하천, 평상이 있어 쉴 수 있는 곳 가평계곡 계곡 관광지 명지계곡유원지 물놀이 서울근교 하천' metadata={'region': '경기도', 'place': '호수유원지', 'text': '명지계곡 물이 흐르는 하천, 평상이 있어 쉴 수 있는 곳 가평계곡 계곡 관광지 명지계곡유원지 물놀이 서울근교 하천'}
[(Document(metadata={'place': '호수유원지', 'region': '경기도', 'text': '명지계곡 물이 흐르는 하천, 평상이 있어 쉴 수 있는 곳 가평계곡 계곡 관광지 명지계곡유원지 물놀이 서울근교 하천'}, page_content='명지계곡 물이 흐르는 하천, 평상이 있어 쉴 수 있는 곳 가평계곡 계곡 관광지 명지계곡유원지 물놀이 서울근교 하천'), 0.4963289499282837), (Document(metadata={'region': '부산광역시', 'place': '달맞이동산', 'text': '바다를 한눈에 내려다 볼 수 있는 달맞이 공원 가족과함께 걷기좋은 관광지 달맞이길공원 부산가볼만한곳 산책로 해운대공원'}, page_content='바다를 한눈에 내려다 볼 수 있는 달맞이 공원 가족과함께 걷기좋은 관광지 달맞이길공원 부산가볼만한곳 산책로 해운대공원'), 0.49643152952194214), (Document(metadata={'text': '다양한 볼거리가 있는 연안부두 경치좋은곳 관광지 남녀노소 산책하기 산책하기좋은곳 수도권 아이와함께', 'place': '연안부두', 'region': '인천광역시'}, page_content='다양한 볼거리가 있는 연안부두 경치좋은곳 관광지 남녀노소 산책하기 산책하기좋은곳 수도권 아이와함께'), 0.5065575242042542), (Document(metadata={'text': '철새가 나는 천수만을 가르는 길 간월도 관광지 서산간척지 서산방조제 자연 조류탐사 천수만 철새도래지', 'region': '충청남도', 'place

In [11]:
responses = vectorstore.similarity_search(query,k=10)
for i in range(len(responses)):
    print(f'{i+1}번째' )
    print(f'(장소이름: {responses[i].metadata["place"]}')
    print(f'태그 : {responses[i].metadata["text"]}')
    print(f'지역 : {responses[i].metadata["region"]}')

1번째
(장소이름: 호수유원지
태그 : 명지계곡 물이 흐르는 하천, 평상이 있어 쉴 수 있는 곳 가평계곡 계곡 관광지 명지계곡유원지 물놀이 서울근교 하천
지역 : 경기도
2번째
(장소이름: 달맞이동산
태그 : 바다를 한눈에 내려다 볼 수 있는 달맞이 공원 가족과함께 걷기좋은 관광지 달맞이길공원 부산가볼만한곳 산책로 해운대공원
지역 : 부산광역시
3번째
(장소이름: 연안부두
태그 : 다양한 볼거리가 있는 연안부두 경치좋은곳 관광지 남녀노소 산책하기 산책하기좋은곳 수도권 아이와함께
지역 : 인천광역시
4번째
(장소이름: 서산A지구방조제
태그 : 철새가 나는 천수만을 가르는 길 간월도 관광지 서산간척지 서산방조제 자연 조류탐사 천수만 철새도래지
지역 : 충청남도
5번째
(장소이름: 월문온천 휴양지
태그 : 달빛이 대문으로 비치는 온천 가족과함께 관광지 사우나 수도권 수도권온천 연인과함께 온천 온천&스파
지역 : 경기도
6번째
(장소이름: 묵호등대
태그 : 시원한 동해바다가 한 눈에 들어오는 곳 강원권 강원도_여행지_추천 강원웰니스관광지 관광지 동해웰니스관광
지역 : 강원특별자치도
7번째
(장소이름: 철원 철새도래지 (철원평야)
태그 : 천연기념물로 지정된 세계적인 철새도래지 강원권 관광지 두루미탐조 자연 천연기념물 철새도래지 철원평야
지역 : 강원특별자치도
8번째
(장소이름: 국립인천해양박물관
태그 : 바다로 열어가는 미래 공간 인천해양박물관 가볼래터 국립해양박물관 문화시설 아이와함께 여행구독 월미도 인천가볼만한곳
지역 : 인천광역시
9번째
(장소이름: 능인선원
태그 : 대한민국 최고의 포교사찰 관광지 능인선원 대한불고조계종 불교 불교문화 사찰 사찰여행 수도권
지역 : 서울특별시
10번째
(장소이름: 신륵사(여주)
태그 : 아름다운 강변에 위치한 천년고찰 경치좋은곳 관광지 불교 불교문화 불교사찰 사찰 사찰여행 수도권
지역 : 경기도


In [52]:
responses[0].metadata

{'place': '연안부두',
 'text': '다양한 볼거리가 있는 연안부두 경치좋은곳 관광지 남녀노소 산책하기 산책하기좋은곳 수도권 아이와함께',
 'region': '인천광역시'}

In [2]:
# 질문 입력 -> 질문 임베딩 -> collection에 전달 -> 상위 결과 5개 뽑기
question = '가족들과 가기 좋은 캠핑장 알려줘'

embedded_question = embedding_model.encode(question).tolist()

top_n = 5
context = collection.query(
    query_embeddings= embedded_question,
    n_results = top_n
)

# 확인용 코드
print(f'Q: {question}')
for i in range(top_n):
    print(context)
    print(f"장소 이름 : {context['metadatas'][i][0]['place']}")
    print(f"주소 : {context['metadatas'][i][0]['주소']}")
    print(f"태그 : {context['metadatas'][i][0]['태그']}")
    print()

NameError: name 'embedding_model' is not defined

In [ ]:
!pip install langchain-openai

In [ ]:
# sllm 모델 생성
from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "0lipa/SEED-1.5B-travel-finetuned"
# sllm = AutoModel.from_pretrained("0lipa/SEED-1.5B-travel-finetuned", torch_dtype="auto")
# sllm = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
# tokenizer = AutoTokenizer.from_pretrained(model_name)

from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.5
)

In [ ]:
# 프롬프트 생성 -> sllm 응답 요청 

# 채팅 로그 수정할 수 있는 함수 설정
def modify_chat_log(chatting_log,message):
    if len(chatting_log)% 2 == 0:
        chatting_log.append({"role": "assistant", "content": message})
    else:
        chatting_log.append({'role':'user','content':message})
    return chatting_log

prompt = f'{question} \n 다음 정보를 활용하여 간략하게 10문장 이내로 출력하시오.{context}'

chat_history = [
    {"role": "system", "content": "너는 대한민국 관광지를 친절하게 알려주는 챗봇이야. 질문에 친절하게 간략하게 답변해줘. 단, 문맥에 없는 정보는 만들어내지마."}
]

chat_history = modify_chat_log(chat_history,prompt)

inputs = tokenizer.apply_chat_template(chat_history, add_generation_prompt=True, return_dict=True, return_tensors="pt")
inputs = inputs.to("cuda")
output_ids = sllm.generate(
    **inputs,
    max_length=1024,
    stop_strings=["<|endofturn|>", "<|stop|>"],
    tokenizer=tokenizer
    )
print(tokenizer.batch_decode(output_ids)[0])

---
### Chromadb
- embedding model : 'all-MiniLM-L6-v2'
- Document(page_content = 태그와 짧은 설명 , metadata = 나머지 정보들)

In [ ]:
# pd 생성 후 태그랑 short_info 합쳐서 임베딩하기

df = pd.read_csv('찐막 전국 데이터.csv')

documents = []
for idx in range(len(df)):
    data = ''
    if df.iloc[idx]['태그'] is not None and df.iloc[idx]['short_info'] is not None:
        data = df.iloc[idx]['태그'] + " " + df.iloc[idx]['short_info']
    elif df.iloc[idx]['태그'] is not None and df.iloc[idx]['short_info'] is None:
        data = df.iloc[idx]['태그']
    elif df.iloc[idx]['태그'] is None and df.iloc[idx]['short_info'] is not None:
        data = df.iloc[idx]['short_info']
    else:
        pass
    documents.append(data)

embedded_docs = embedding_model.encode(documents).tolist()

In [ ]:
# 메타 데이터 생성

def remove_nonwords(text):
    if type(text)!= float :
        text = text.replace(':)',"")    # :) 제거
        text = text.replace("※","")
        text = text.replace('-','')
        text = text.replace('\n더보기','')      # \n 더보기, \n 닫기, \n 제거
        text = text.replace('\n닫기','')
        text = text.replace('\r\n',' ')
        text = text.replace('\n',' ')
        text = text.replace('\r', ' ')
        text = re.sub('[^a-zA-Z가-힣\'"· 0-9.,()㎡[0-9]~[0-9]]','',text) # 숫자와 숫자 사이에 있는 ~ 남겨야함
    else:
        pass
    return text     # 제거한 텍스트 반환

metadata=[]
for idx in range(len(df)):
    meta_data = {}
    row = df.iloc[idx][df.iloc[idx].notna()]
    cols = list(row.index)   # None이 아닌 컬럼들 == row.index 

    for col in cols:     
        modified_data = remove_nonwords(row[col])
        meta_data[col] = modified_data
    metadata.append(meta_data)

In [ ]:
# 컬렉션에 추가
collection.add(
    ids = [str(i) for i in range(len(documents))],
    embeddings = [embedded_docs],
    documents = documents,      # 오류나면 빼기
    metadatas = metadata
)

In [ ]:
# 질문 입력 -> 질문 임베딩 -> collection에 전달 -> 상위 결과 5개 뽑기
question = '가족들과 가기 좋은 캠핑장 알려줘'

embedded_question = embedding_model.encode(question).tolist()

top_n = 5
context = collection.query(
    query_embeddings= embedded_question,
    n_results = top_n
)

# 확인용 코드
print(f'Q: {question}')
for i in range(top_n):
    print(context)
    print(f"장소 이름 : {context['metadatas'][i][0]['place']}")
    print(f"주소 : {context['metadatas'][i][0]['주소']}")
    print(f"태그 : {context['metadatas'][i][0]['태그']}")
    print()

In [ ]:
# sllm 모델 생성
from transformers import AutoModel

model_name = "0lipa/SEED-1.5B-travel-finetuned"
# sllm = AutoModel.from_pretrained("0lipa/SEED-1.5B-travel-finetuned", torch_dtype="auto")
sllm = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# 프롬프트 생성 -> sllm 응답 요청 

# 채팅 로그 수정할 수 있는 함수 설정
def modify_chat_log(chatting_log,message):
    if len(chatting_log)% 2 == 0:
        chatting_log.append({"role": "assistant", "content": message})
    else:
        chatting_log.append({'role':'user','content':message})
    return chatting_log

prompt = f'{question} \n 다음 정보를 활용하여 간략하게 10문장 이내로 출력하시오.{context}'

chat_history = [
    {"role": "system", "content": "너는 대한민국 관광지를 친절하게 알려주는 챗봇이야. 질문에 친절하게 간략하게 답변해줘. 단, 문맥에 없는 정보는 만들어내지마."}
]

chat_history = modify_chat_log(chat_history,prompt)

inputs = tokenizer.apply_chat_template(chat_history, add_generation_prompt=True, return_dict=True, return_tensors="pt")
inputs = inputs.to("cuda")
output_ids = sllm.generate(
    **inputs,
    max_length=1024,
    stop_strings=["<|endofturn|>", "<|stop|>"],
    tokenizer=tokenizer
    )
print(tokenizer.batch_decode(output_ids)[0])

----
# Retriever 이용

In [ ]:
# Document로 저장

from langchain.text_splitter import RecursiveCharacterTextSplitter      # 재귀적으로 text splitter  큰 chunk -> 작은 chunk 로 분할할 chunk size 지정
import preprocess_functions as pf

documents = pf.make_tag_page_content('찐막 전국 데이터.csv')    # 태그와 짤막한 설명을 같이 page_content에 넣어주는 경우
documents

In [ ]:
# 임베딩 모델 생성 및 vectorstore 생성
from langchain_chroma.vectorstores import Chroma
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
vector_store = Chroma.from_documents(documents, embedding_model)

In [32]:
# 2. Retriever를 활용한 검색
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

question = '가족들과 가기 좋은 캠핑장 알려줘'

# retriever = vectorstore.as_retriever(
#     search_type='similarity',
#     search_kwargs = {'k':10}
# )

# retriever_result = retriever.batch([question])
# retriever_result


metadata_field_info = [
    AttributeInfo(
        name='text',
        description='해당 장소에 대한 간략한 설명. 어떤 종류의 장소인지 알려주는 용도.',
        type='string'
    ),
    AttributeInfo(
        name='region',
        description='해당 장소가 위치한 지역명.',
        type='string'
    ),
    AttributeInfo(
        name='place',
        description='해당 장소의 이름',
        type='string'
    )
]

document_content = '전국 관광지들의 정보가 저장되어 있다.'

retriever = SelfQueryRetriever.from_llm(
    llm=sllm,
    vectorstore=vectorstore,
    document_contents=document_content,
    metadata_field_info=metadata_field_info
)

retriever_result = retriever.batch([question])
retriever_result

ImportError: Cannot import lark, please install it with 'pip install lark'.

In [14]:
# 프롬프트 템플릿 생성

from langchain_core.prompts import ChatPromptTemplate

chatting_log = [
    ('system', '너는 대한민국 관광지를 친절하게 알려주는 챗봇이야. 질문에 친절하게 간략하게 답변해줘. 단, 문맥에 없는 정보는 만들어내지마.'),
    ('user', '''어린이의 질문에 context만을 이용해 답변하세요.
context에서 확인할 수 없는 질문이라면 모른다고 답변해야 합니다.
최종 응답에는 참조한 context에 대한 정보를 추가해 주세요.

     질문: {query}
     context: {context}
''')
]

prompt_tpl = ChatPromptTemplate(chatting_log)
# prompt_tpl.invoke({'query': query, 'context': retriever_result})

In [23]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [34]:
# sllm 모델 생성
from transformers import AutoModel
from transformers.models import AutoModelForCausalLM, AutoTokenizer

# model_name = "0lipa/SEED-1.5B-travel-finetuned"
# sllm = AutoModel.from_pretrained("0lipa/SEED-1.5B-travel-finetuned", torch_dtype="auto")
# model_name = 'Saxo/Linkbricks-Horizon-AI-Korean-llama-3.1-sft-dpo-8B'
model_name = '0lipa/Linkbricks-Horizon8.0B-travel-finetuned'
sllm = AutoModel.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

ValueError: Unrecognized model in 0lipa/Linkbricks-Horizon8.0B-travel-finetuned. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: aimv2, aimv2_vision_model, albert, align, altclip, apertus, arcee, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, bitnet, blenderbot, blenderbot-small, blip, blip-2, blip_2_qformer, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, cohere2_vision, colpali, colqwen2, conditional_detr, convbert, convnext, convnextv2, cpmant, csm, ctrl, cvt, d_fine, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deepseek_v2, deepseek_v3, deepseek_vl, deepseek_vl_hybrid, deformable_detr, deit, depth_anyth

In [ ]:
# 체인 생성
from langchain_core.output_parsers import StrOutputParser

chain = prompt_tpl | sllm | StrOutputParser()

In [ ]:
retriever_result = retriever.batch([query])
response = chain.invoke({'query': question, 'context': retriever_result})
print(response)

-----

In [5]:
# pd 생성 후 태그만 임베딩하기
import pandas as pd

df = pd.read_csv('찐찐찐찐막 전국 데이터.csv')

C:\Users\brain\AppData\Local\Temp\ipykernel_10352\3822900607.py:4: DtypeWarning: Columns (2,55,57,58,59,60,63,64,67,68,69,70,72) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('찐찐찐찐막 전국 데이터.csv')


In [6]:
# 메타 데이터 생성
metadata=[]
for idx in range(len(df)):
    meta_data = {}
    meta_data['text'] = df.iloc[idx]['태그']
    meta_data['region'] = df.iloc[idx]['주소'].split()[0]
    meta_data['place'] = df.iloc[idx]['place']
    metadata.append(meta_data)

In [7]:
metadata, len(metadata)

([{'text': '조선의 근본을 상징하는 국왕의 제사터 관광지 문화유적 문화유적돌아보기 서울 서울관광지 서울역사관광 역사관광',
   'region': '광주광역시',
   'place': '사직단'},
  {'text': '그날의 역사와 의의를 자세히 알 수 있는 곳 5.18 518민주화운동기록관 5·18민주화운동기록관 광주 광주여행 기록관',
   'region': '광주광역시',
   'place': '5·18민주화운동 기록관'},
  {'text': '봄을 먼저 맞이하는 곳 광주가볼만한곳 광주꽃구입 광주꽃단지 광주꽃시장 광주나무구입 광주여행',
   'region': '광주광역시',
   'place': '상무화훼단지'},
  {'text': '생산자와 소비자를 보호하는 호남 최대의 도매시장 농수산물 농수산물검사소 농수산물경매 도매시장 서부농수산물도매시장',
   'region': '광주광역시',
   'place': '서부농수산물도매시장'},
  {'text': '코스모스와 억새 물결이 일렁이는 공원 관광지 광주가볼만한곳 광주코스모스 산동교 산동교친수공원 유채꽃명소',
   'region': '광주광역시',
   'place': '산동교친수공원'},
  {'text': '우리나라 최초의 폐선부지 공원 2022친환경추천여행지 관광지 광주 광주_여행지_추천 광주가볼만한곳',
   'region': '광주광역시',
   'place': '푸른길분수공원'},
  {'text': '일본통신사 행차를 기록한 조형선생을 기리기 위한 비 관광지 광주취병조형유허비 교과서속여행 문화재 역사 역사공부 역사를품은곳',
   'region': '광주광역시',
   'place': '취병조형유허비'},
  {'text': '우리나라 무덤의 발달사와 한일관계의 역사를 보여주는 삼국시대 고분 관광지 광주고분 광주월계동장고분 신라고분 역사 역사공부 역사관광지',
   'region': '광주광역시',
   'place': '월계동장고분'},
  {'text': '석불입상과 함께

In [8]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma

documents = [df.iloc[idx]['태그'] for idx in range(len(df))]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
docs = text_splitter.create_documents(documents,metadata)
doc = text_splitter.split_documents(docs)

embedding_model = HuggingFaceBgeEmbeddings(model_name = 'all-MiniLM-L6-v2')

vectorstore = Chroma.from_documents(docs, embedding_model)

In [10]:
!pip install langchain-openai

   ---------------------------------------- 0.0/948.4 kB ? eta -:--:--
   --------------------------------------- 948.4/948.4 kB 42.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/884.3 kB ? eta -:--:--
   --------------------------------------- 884.3/884.3 kB 38.9 MB/s eta 0:00:00

  Attempting uninstall: openai

    Found existing installation: openai 1.102.0

   ------------- -------------------------- 1/3 [openai]
   ------------- -------------------------- 1/3 [openai]
    Uninstalling openai-1.102.0:
   ------------- -------------------------- 1/3 [openai]
      Successfully uninstalled openai-1.102.0
   ------------- -------------------------- 1/3 [openai]
   ------------- -------------------------- 1/3 [openai]
   ------------- -------------------------- 1/3 [openai]
   ------------- -------------------------- 1/3 [openai]
   ------------- -------------------------- 1/3 [openai]
   ------------- -------------------------- 1/3 [openai]
   ------------- -------

In [ ]:
# 2. Retriever를 활용한 검색
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

question = '가족들과 가기 좋은 캠핑장 알려줘'

# retriever = vectorstore.as_retriever(
#     search_type='similarity',
#     search_kwargs = {'k':10}
# )

# retriever_result = retriever.batch([question])
# retriever_result


metadata_field_info = [
    AttributeInfo(
        name='text',
        description='해당 장소에 대한 간략한 설명. 어떤 종류의 장소인지 알려주는 용도.',
        type='string'
    ),
    AttributeInfo(
        name='region',
        description='해당 장소가 위치한 지역명.',
        type='string'
    ),
    AttributeInfo(
        name='place',
        description='해당 장소의 이름',
        type='string'
    )
]

document_content = '전국 관광지들의 정보가 저장되어 있다.'


from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.1
)

retriever = SelfQueryRetriever.from_llm(
    llm=model,
    vectorstore=vectorstore,
    document_contents=document_content,
    metadata_field_info=metadata_field_info
)

retriever_result = retriever.batch([question])
retriever_result

[[Document(metadata={'place': '장흥 참숯가마', 'text': '새로운 찜질체험 관광지', 'region': '경기도'}, page_content='새로운 찜질체험 관광지'),
  Document(metadata={'place': '장흥 참숯가마', 'text': '새로운 찜질체험 관광지', 'region': '경기도'}, page_content='새로운 찜질체험 관광지'),
  Document(metadata={'region': '충청남도', 'text': '배롱나무꽃과 연못이 아름다운 향교 관광지 노성향교 논산 논산노성향교 논산노성향교 논산향교 역사', 'place': '노성향교'}, page_content='배롱나무꽃과 연못이 아름다운 향교 관광지 노성향교 논산 논산노성향교 논산노성향교 논산향교 역사'),
  Document(metadata={'region': '충청남도', 'text': '배롱나무꽃과 연못이 아름다운 향교 관광지 노성향교 논산 논산노성향교 논산노성향교 논산향교 역사', 'place': '노성향교'}, page_content='배롱나무꽃과 연못이 아름다운 향교 관광지 노성향교 논산 논산노성향교 논산노성향교 논산향교 역사')]]